<a href="https://colab.research.google.com/github/T3LNET/DeepLearning/blob/main/Copy_of_gray_to_rgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download dataset

In [ ]:
!pip install kaggle

In [ ]:
!cp kaggle.json /root/.kaggle

In [ ]:
!kaggle datasets download -d benjaminbirchhansen/img-net-10percent

In [ ]:
!unzip img-net-10percent.zip -d img-net-10

## imports  

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import  os
import cv2

## MODEL conv

In [ ]:
model = models.Sequential()
model.add(layers.Input(shape=(700, 525, 1)))

model.add(layers.Conv2D(32, (4, 4), padding='same'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(64, (4, 4), padding='same'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(128, (4, 4), padding='same'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(256, ((4, 4))))
model.add(layers.MaxPool2D((2, 2)))

#-----------

model.add(layers.Conv2D(128, (4, 4), padding = 'same'))
model.add(layers.UpSampling2D(2))

model.add(layers.Conv2D(64, (4, 4), padding= 'same'))
model.add(layers.UpSampling2D(2))

model.add(layers.Conv2D(32, (4, 4), padding= 'same'))
model.add(layers.UpSampling2D(2))

model.add(layers.Conv2D(10, (4, 4), padding= 'same'))
model.add(layers.UpSampling2D(2))

model.add(layers.Conv2D(3, (4, 4), padding= 'same'))
model.add(layers.Activation('sigmoid'))

In [ ]:
# model.output_shape()
model.compile('sgd', 'mean_squared_error')
model.summary()

## model VGG-16

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D
import pandas as pd
import os
from sklearn.utils import shuffle

def build_model(input_shape):
    inputs = Input(shape=input_shape)

    # Custom VGG-16-like encoder for grayscale input
    encoder = tf.keras.Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
    ])

    encoder_output = encoder(inputs)

    # Decoder
    decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder_output)
    decoder = UpSampling2D((2, 2))(decoder)
    decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
    decoder = UpSampling2D((2, 2))(decoder)
    decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
    decoder = UpSampling2D((2, 2))(decoder)
    decoder = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(decoder)

    model = tf.keras.Model(inputs, decoder)
    return model

input_shape = (224, 224, 1)  # Grayscale image input
model = build_model(input_shape)
model.summary()

<details>
<summary><b>TFRecord</b></summary>
<code>
# DataPath
data_path = 'PATH'
with tf.Session() as sess:
    feature = {
        'train/originals': tf.FixedLenFeature([], tf.string),

        'train/labels': tf.FixedLenFeature([], tf.string)
         }

    filename_queue = tf.train.string_input_producer([data_path],num_epochs=None,shuffle=True,seed=1000000)

    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(serialized_example, features=feature)

    originals = tf.decode_raw(features['train/originals'], tf.uint8)
    labels = tf.decode_raw(features['train/labels'], tf.float64)

    originals = tf.reshape(originals, [224, 224, 3])
    originals = tf.cast(originals,tf.float32)
    labels = tf.cast(labels, tf.float32)


    labels = tf.reshape(labels,[1])
    # init = tf.global_variables_initializer()
    # sess.run(init)
    # tf.train.start_queue_runners()

    original , label = tf.train.shuffle_batch([originals, labels],  batch_size=10, capacity=30, num_threads=1, min_after_dequeue=10)

    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)


    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    counter = 0
    for batch_index in range(500000):
        counter +=1
        print(counter)
        o,lab = sess.run([original,label])
        print(o.shape)
        print(lab[1])

        # cv2.imshow('salam', np.array(o[1], dtype='uint8'))
        # # cv2.waitKey(100)

    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)
    sess.close()
</code>

<br>
<br>

<code>
data_path = 'PATH'

def intt64Feature(value):
    return  tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def bytesFeature(value):
    return  tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def createDataSetTFRecord(data_path'''mnist_path''',tf_path):
    data_splits = ["train","test","validation"]
    # datasets = mnist.read_data_sets(mnist_path,dtype=tf.uint8,reshape=False,validation_size=1032)

for d in range(len(data_splits)):
        cur_split = data_splits[d]
        print("saving "+cur_split)
        dataset = datasets[d]
        file_name = os.path.join(tf_path,cur_split+".tfrecord")
        print(file_name)
        writer = tf.python_io.TFRecordWriter(file_name)

        images_count = dataset.images.shape[0]
        for index in range(images_count) :
            cur_img =dataset.images[index]
            image = cur_img.tostring()
            example = tf.train.Example(features=tf.train.Features(
                feature={
                    'height':intt64Feature(dataset.images.shape[1]),
                    'width':intt64Feature(dataset.images.shape[2]),
                    'depth':intt64Feature(dataset.images.shape[3]),
                    'label':intt64Feature(dataset.labels[index]),
                    "raw_image":bytesFeature(image)
                }
            ))
            writer.write(example.SerializeToString())
        writer.close()

#Load dataset
def loadDataSetFromTFRecord(file_name):

    data_iterator = tf.python_io.tf_record_iterator(file_name)

    while True:
        try:
            example_serialized = next(data_iterator)
            example = tf.train.Example()
            example.ParseFromString(example_serialized)

            width = example.features.feature['width'].int64_list.value[0]
            height = example.features.feature['height'].int64_list.value[0]
            depth = example.features.feature['depth'].int64_list.value[0]
            label = example.features.feature['label'].int64_list.value[0]
            image = example.features.feature['raw_image'].bytes_list.value

            flat_image = np.fromstring(image[0],np.uint8)
            reshaped_img = flat_image.reshape((height,width,-1))
            cv2.imshow("view",reshaped_img)
            cv2.waitKey(0)
        except tf.errors.OutOfRangeError:
            break
</code>
</details>

## TFRECORD

In [ ]:
# One class dataset
import os
import tensorflow as tf
import numpy as np
from PIL import Image

# Define the path to your single-class dataset and output TFRecord file
dataset_path = 'PATH'
tfrecord_filename = 'single_class_dataset.tfrecord'

# Function to encode image as a feature
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# Function to encode integer label as a feature
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Open a TFRecord writer
writer = tf.io.TFRecordWriter(tfrecord_filename)

# Loop through each image in the single-class dataset
for image_filename in os.listdir(dataset_path):
    image_path = os.path.join(dataset_path, image_filename)
    image = np.array(Image.open(image_path))  # Load and preprocess image
    image_raw = image.tostring()  # Convert image to raw bytes

    # Assuming the class label is 0 for a single class dataset
    label = 0

    # Create a feature
    feature = {
        'image_raw': _bytes_feature(image_raw),
        'label': _int64_feature(label)
    }

    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize the example to a string and write to the TFRecord file
    writer.write(example.SerializeToString())

# Close the TFRecord writer
writer.close()

# Parse TFRecord dataset for training
def parse_function(example_proto):
    features = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    parsed_features = tf.io.parse_single_example(example_proto, features)

    image = tf.image.decode_image(parsed_features['image_raw'])
    image = preprocess_image(image)  # Define your preprocessing function

    label = parsed_features['label']

    return image, label

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset([tfrecord_filename])
dataset = dataset.map(parse_function)
dataset = dataset.shuffle(buffer_size=500)
dataset = dataset.batch(batch_size=32)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)